# DESaster Simulation Set Up Template

## Required Modules

In [25]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from simpy import Resource, Container, FilterStore
from simpy.util import start_delayed
import pandas as pd
import numpy as np
from desaster import entities, capitals, request, io, movement, search, rebuild

## Input Data

In [26]:
scenario_file = '../inputs/scenario_test_renters.xlsx'

# Create Pandas dataframe of attribute data for all households to be modeled in the simulation
households_df = pd.read_excel(scenario_file, sheetname='households')

# Create Pandas dataframe of attribute data for all vacant homes (housing stock) to be modeled in the simulation
vacant_stock_df = pd.read_excel(scenario_file, sheetname='housing_stock')

# Set input data for all human capital types, as dict or Pandas Series
# .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]

# Set input data for all financial capital types, as dict or Pandas Series
financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]

In [27]:
households_df

,Occupant,Owner,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure Pref,Tenure,Occupancy Pref
0,Alfred,Landlord,30000,10000,0,62 That St,Mobile Home,100,1,1,700,1920,100000,None,Own,Rent,Single Family Dwelling
1,Bruce,Bruce,100000,1000000,10000000,720 This Rd,Single Family Dwelling,100000,5,5,5000,1920,10000000,Slight,Own,Own,Single Family Dwelling
2,Selena,Landlord,10000,100,0,1001 Other Ave,Multi Family Dwelling,10,0,1,250,1960,10000,Complete,Rent,Rent,Multi Family Dwelling
3,Fish,Fish,50000,1000,550000,26000 Out There Lane,Single Family Dwelling,2000,4,2,2000,2010,800000,Moderate,Own,Own,Single Family Dwelling


In [28]:
vacant_stock_df.head()

,Occupant,Owner,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure
0,NaN,NaN,100 New Ave,Mobile Home,100,1,1,700,1920,99999,Complete,Own
1,NaN,NaN,101 New Ave,Single Family Dwelling,100000,6,5,5000,1920,9999,Complete,Own
2,NaN,NaN,102 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,Complete,Own
3,NaN,NaN,103 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,Complete,Own
4,NaN,NaN,104 New Ave,Mobile Home,100,1,1,700,1920,100000,Complete,Own


In [29]:
human_cap_data

Type
Contractors             40
Engineers                1
FEMA Processors         40
Inspectors             100
Insurance Adjusters     40
Loan Processors         40
Permit Processors      100
Name: Quantity, dtype: int64

In [30]:
financial_cap_data

Type
Building Materials     2000000
FEMA Aid              35000000
Name: Quantity, dtype: int64

## Simulation Initiation

__Set simulation environment__

In [31]:
simulation = simpy.Environment()

__Indicate whether want to keep track of the stories of each entity (household) in the simulation.__

In [32]:
write_story = True

__Populate the simulation with the input data specified above.__

In [33]:
financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) 

human_capital = capitals.HumanCapital(simulation, human_cap_data)

In [34]:
occupied_stock = FilterStore(simulation)
occupied_stock.items # Nothing in the FilterStore

households = entities.importHouseholds(simulation, occupied_stock, households_df, write_story)

In [35]:
vacant_stock = capitals.importHousingStock(simulation, vacant_stock_df)

__Write a master process that combines process and functions from search, rebuild, and request modules.__

In [36]:
def master_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 365  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
                return
        
        # If home is completely damaged, search for a new home to purchase.
        if entity.residence.damage_state == 'Complete':
            
            home_patience = 550  # days until give up the search for a new home

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, vacant_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
               
   

In [37]:
# Initiate a master process for each household to be modeled in the simulation
for i in range(len(households)):
    simulation.process(master_process(simulation, human_capital, financial_capital, households[i], write_story))

__Can do some cool stuff with the vacant housing stock.__

In [38]:
# Do inspections on all of the vacant homes in the housing stock
for home in vacant_stock.items:
    simulation.process(request.inspection(simulation, human_capital, home))

# Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# with probability = fix_probability
fix_probability = 1.0
fix_schedule = 100

# start_delayed(simulation, rebuild.stock(simulation, vacant_stock, fix_probability), fix_schedule)

In [39]:
simulation.run()

## Outputs

__Summary statistics__

In [40]:
num_undamaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_gave_up_home_search = 0

for household in households:
    if household.money_search_start == 0.0: num_undamaged += 1
    if household.home_get > 0.0: num_rebuilt += 1
    if household.gave_up_money_search: num_gave_up_money_search += 1
    if household.home_search_stop > 0.0: num_relocated += 1
    if household.gave_up_home_search: num_gave_up_home_search += 1
        
print('{0} out of {1} households suffered no damage to their homes.\n'.format(num_undamaged, len(households)),
      '{0} out of {1} households rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(households)),
        '{0} out of {1} households gave up searching for money.\n'.format(num_gave_up_money_search, len(households)),
        '{0} out of {1} households decided to find a new home.\n'.format(num_relocated, len(households)),
        '{0} out of {1} households gave up searching for a home.'.format(num_gave_up_home_search, len(households))
      )

1 out of 4 households suffered no damage to their homes.
 2 out of 4 households rebuilt or repaired their damaged home.
 0 out of 4 households gave up searching for money.
 0 out of 4 households decided to find a new home.
 1 out of 4 households gave up searching for a home.


__Household stories__

In [41]:
households[0].story

['Alfred lives in a 1 bedroom Mobile Home at 62 That St worth $100,000. ',
 "Alfred's house was inspected 10 days after the event and suffered $0 of damage."]

In [42]:
households[1].story

['Bruce lives in a 5 bedroom Single Family Dwelling at 720 This Rd worth $10,000,000. ',
 "Bruce's house was inspected 10 days after the event and suffered $200,000 of damage.",
 'Bruce submitted an insurance claim 10 days after the event. ',
 'Bruce received a $200,000 insurance payout 25 days after the event. ',
 'It took Bruce 15 days to receive enough financial assistance and now has $1,200,000 to rebuild. ',
 'Bruce received an engineering assessment 50 days after the event. ',
 'Bruce received permit approval 85 days after the event. ',
 "Bruce's home was repaired 87 days after the event, taking 2 days to repair. "]

In [43]:
households[2].story

['Selena lives in a 0 bedroom Multi Family Dwelling at 1001 Other Ave worth $10,000. ',
 "Selena's house was inspected 10 days after the event and suffered $10,000 of damage.",
 'Selena submitted a request to FEMA 10 days after the event. ',
 'Selena received $10,000 from FEMA 30 days after the event. ',
 'It took Selena 20 days to receive enough financial assistance and now has $10,100 to rebuild. ',
 'Selena started searching for a Multi Family Dwelling with a value under $10,000 30 days after the event. ',
 'On day 580, after a 550 day search, Selena gave up looking for a new home in the local area. ']

In [44]:
households[3].story

['Fish lives in a 4 bedroom Single Family Dwelling at 26000 Out There Lane worth $800,000. ',
 "Fish's house was inspected 10 days after the event and suffered $80,000 of damage.",
 'Fish submitted an insurance claim 10 days after the event. ',
 'Fish received a $80,000 insurance payout 25 days after the event. ',
 'It took Fish 15 days to receive enough financial assistance and now has $81,000 to rebuild. ',
 'Fish received an engineering assessment 75 days after the event. ',
 'Fish received permit approval 110 days after the event. ',
 "Fish's home was repaired 140 days after the event, taking 30 days to repair. "]